In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Inshal Munaf\\VsCodeProjects\\Kidney_disease_Classification_MLOPS\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Inshal Munaf\\VsCodeProjects\\Kidney_disease_Classification_MLOPS'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class PrepareBaseModelConfig:
    root_dir : Path
    base_model_path:Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights:str
    params_classes:int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
    ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config =self.config.prepare_base_model

        create_directories([config.root_dir])
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = config.root_dir,
            base_model_path= config.base_model_path,
            updated_base_model_path=config.updated_base_model_path,
            params_image_size=self.params.IMAGE_SIZE,
            params_classes=self.params.CLASSES,
            params_include_top=self.params.INCLUDE_TOP,
            params_learning_rate=self.params.LEARNING_RATE,
            params_weights=self.params.WEIGHTS
        )

        return prepare_base_model_config
